## VacationPy Homework

Code takes excel csv from WeatherPy and selects locations that meet specified criteria for temperature, humidity, %cloud and 
wind speed.



In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import gmaps
from config import gkey
import requests
import json




In [2]:
#defining ideal conditions
temp_max = 35 #in degree C
temp_min = 25 #in degree C
humidity_max=45 #in %
cloud_max=50 #in %
wind_max=7 #in meters/sec

In [3]:
#Importing data.  Data has been transferred from WeatherPy to subdirectory
filename = "Data_and_Output/weather.csv"
destination_df = pd.read_csv(filename, dtype = str)
destination_df = destination_df.astype({"Lat":"float64","Long":"float64", "Temp":"float64", "Humidity":"float64",
                                        "%Cloud":"float64","Wind_Speed":"float64"})
start_num=destination_df["Temp"].count()

In [4]:
#creating heat map of humidity
gmaps.configure(gkey)

locations = destination_df[["Lat", "Long"]]
humidity = destination_df["Humidity"]


hum_max = humidity.max()
figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '2px solid black',
    'padding': '4px'
}

#creating details of map.  Zoom and center are necessary to prevent odd scaling of map
fig = gmaps.figure(layout = figure_layout, map_type="TERRAIN", center = [0,0], zoom_level =1)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating = False, max_intensity = hum_max,  
                                 point_radius=4)
heat_layer.gradient = ["aqua","green", "yellow","red"]  #color codes from (https://www.w3.org/TR/css-color-3/#html4)
heat_layer.opacity = 0.3
fig.add_layer(heat_layer)

print("Heat Map Showing Humidity Variation at 500 Locations Chosen Randomly")

fig

Heat Map Showing Humidity Variation at 500 Locations Chosen Randomly


Figure(layout=FigureLayout(border='2px solid black', height='600px', padding='4px', width='600px'))

In [5]:
#filtering data based on climate conditions
mid_num = destination_df["Temp"].count()
destination_df = destination_df.loc[(destination_df["Temp"]<=temp_max)&(destination_df["Temp"]>=temp_min)]
destination_df = destination_df.loc[(destination_df["Humidity"]<=humidity_max)]
destination_df = destination_df.loc[(destination_df["%Cloud"]<=cloud_max)]
destination_df =destination_df.loc[(destination_df["%Cloud"]<=wind_max)]



In [6]:
#code below finds hotels in cities that meet climate requirements.  In some cases no hotel may be available, so the text
#  "No hotel in 10 km" is included
# assignment originally called for searching within 5km radius.  This tended to give no results.  Increased to 10 km to 
# give some results

destination_df = destination_df.astype({"Lat":"object","Long":"object"})

loc_x = [destination_df.iloc[i][2] for i in range(0,len(destination_df))]
loc_y = [destination_df.iloc[i][3] for i in range(0,len(destination_df))]
loc = [(f"{loc_x[i]} , {loc_y[i]}") for i in range(0,len(loc_x))]

#target_search="Hotel"
target_search = "hotel"
target_radius = 10000
target_type = "hotel"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


m=0 #m to track exceptions if needed
hotel = []
lat = []
long = []
for i in range(0,len(loc)):
            params = {
                   "location":loc[i],
                    "keyword":target_search,
                   "radius":target_radius,
                   "type":target_type,
                   "key":gkey
               }
            try:
                response = requests.get(base_url, params=params)
                data=response.json()
                
            except KeyError:
                m+=1
            if data["status"]!= "ZERO_RESULTS":
                hotel.append(data["results"][0]["name"])
                lat.append((data["results"][0]["geometry"]["location"]["lat"]))
                long.append((data["results"][0]["geometry"]["location"]["lng"]))
                
            else:
                hotel.append("No hotel in 10 km")
                lat.append((loc_x[i]))
                long.append((loc_y[i]))
                
         
            
            

In [7]:
#assembling a dataframe with final information
loc_vacation_df = pd.DataFrame(columns = ["Hotel", "Lat", "Long"])
for i in range(0,len(hotel)):
    if hotel[i]!= "No hotel in 10 km":        
        loc_x = lat[i]
        loc_y = long[i]
        loc_vacation_df = loc_vacation_df.append({"Hotel":hotel[i],"Lat":loc_x,"Long":loc_y},ignore_index = True)



In [8]:
#code below finds locations with hotels from selected list meeting climate conditions
country=[] #list of countries
city = [] #list of cities
length_list = [] #length of results in json to help locate city name

base_url="https://maps.googleapis.com/maps/api/geocode/json?"
for i in range(0,(loc_vacation_df["Hotel"].count())):
    #creating tuple with coordinates
    lat = str(loc_vacation_df.iloc[i][1])
    long = str(loc_vacation_df.iloc[i][2])
    lat_long = f"{lat},{long}"
    
    #assembling URL for query
    query_url = base_url+"latlng="+lat_long+"&key="+gkey
    response = requests.get(query_url).json()
    length = len(response["results"])
    length_list.append(len(response["results"]))
    
    #based on length of "results" in json need to change field for location of city
    if length<9:
        city_p = length - 3
    else:
        city_p = length -4
 
    country.append(response["results"][length-1]["formatted_address"])
    city.append(response["results"][city_p]["address_components"][0]["long_name"])
    
    #print(json.dumps(response, indent=4, sort_keys=True))

In [9]:
#prepping data for markers on maps. 
#need location as a tuple (lat,long) and info as a string (hotel name, city, country)
loc_vacation_df["City"]=city
loc_vacation_df["Country"]=country
print("Selected locations and hotels")
loc_pair=[]
loc_text=[]
for i in range(0,len(loc_vacation_df)):
    loc_pair.append((loc_vacation_df.iloc[i][1],loc_vacation_df.iloc[i][2]))
    loc_text.append(f"{loc_vacation_df.iloc[i][0]} in \n {loc_vacation_df.iloc[i][3]}, {loc_vacation_df.iloc[i][4]}")
    
final_num = loc_vacation_df["Hotel"].count()
loc_vacation_df


Selected locations and hotels


,Hotel,Lat,Long,City,Country
0,Bedouin Valley Eco Lodge,24.965362,34.932579,Marsa Alam,Egypt
1,Hotel Grand Swiss,36.374134,44.201255,Shaqlawa,Iraq
2,هوتل شهرداری,36.911350,66.184928,Mingajik,Afghanistan
3,Araguaia Hotel,-18.217484,-49.736272,Bom Jesus de Goiás,Brazil


In [10]:
#replotting heat map with markers and labels for select hotels
print("Global Humidity Map with Select Hotels in Locations Meeting Following:")
print("")
print(f"Temperature (degree C) between {temp_min} and {temp_max}.")
print(f"Humidity below {humidity_max}%")
print(f"Cloud cover less than {cloud_max}%")
print(f"Wind speed less than {wind_max} m/sec")
print("")
print(f"Started with {start_num} locations and filtered down to {mid_num} locations satisfying climate conditions")
print(f"Then filtered down to {final_num} with hotels in 10km radius.")

markers = gmaps.marker_layer(loc_pair, hover_text = loc_text, display_info_box = True, 
                             info_box_content = loc_text)
fig.add_layer(markers)
fig

Global Humidity Map with Select Hotels in Locations Meeting Following:

Temperature (degree C) between 25 and 35.
Humidity below 45%
Cloud cover less than 50%
Wind speed less than 7 m/sec

Started with 501 locations and filtered down to 501 locations satisfying climate conditions
Then filtered down to 4 with hotels in 10km radius.


Figure(layout=FigureLayout(border='2px solid black', height='600px', padding='4px', width='600px'))